In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/cs4262/EmotionDetection

/content/gdrive/MyDrive/cs4262/EmotionDetection


In [ ]:
pip install -r requirements.txt

In [ ]:
%cd /content/gdrive/MyDrive/cs4262/EmotionDetection/datasets

In [ ]:
ls -l

In [ ]:
!chmod +x getDataset.sh

In [ ]:
! bash getDataset.sh

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score

# Load training data from CSV
train_df = pd.read_csv("/content/gdrive/MyDrive/cs4262/EmotionDetection/datasets/train_data.csv")

# Load testing data from CSV
test_df = pd.read_csv("/content/gdrive/MyDrive/cs4262/EmotionDetection/datasets/test_data.csv")

# Load images and labels from training data
train_images = []
train_labels = []

for index, row in train_df.iterrows():
    image = load_img(row['trainSet'], target_size=(32, 32))
    image = img_to_array(image) / 255.0  # Normalize pixel values
    train_images.append(image)
    train_labels.append(row['trainLabel'])

train_images = np.array(train_images)

# Label encoding
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels = to_categorical(train_labels_encoded)  # One-hot encode labels

# Load images and labels from testing data
test_images = []
test_labels = []

for index, row in test_df.iterrows():
    image = load_img(row['testSet'], target_size=(32, 32))
    image = img_to_array(image) / 255.0  # Normalize pixel values
    test_images.append(image)
    test_labels.append(row['testabel'])

test_images = np.array(test_images)

# Label encoding for test labels
test_labels_encoded = label_encoder.transform(test_labels)
test_labels = to_categorical(test_labels_encoded)  # One-hot encode labels

# Split training data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)


In [ ]:
# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

# Define early stopping callback
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train the model with data augmentation and early stopping
history = model.fit(datagen.flow(train_images, train_labels, batch_size=64, shuffle=True),
                    steps_per_epoch=len(train_images) // 64,
                    epochs=100,
                    validation_data=(val_images, val_labels),
                    callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Plot the loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the accuracy curves
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Make predictions on test data
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Compute accuracy
accuracy = accuracy_score(np.argmax(test_labels, axis=1), predicted_labels)
print(f'Accuracy: {accuracy}')

# Compute precision
precision = precision_score(np.argmax(test_labels, axis=1), predicted_labels, average='weighted')
print(f'Precision: {precision}')

# Compute recall
recall = recall_score(np.argmax(test_labels, axis=1), predicted_labels, average='weighted')
print(f'Recall: {recall}')

# Compute F1 score
f1 = f1_score(np.argmax(test_labels, axis=1), predicted_labels, average='weighted')
print(f'F1 score: {f1}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Generate a random index to select a random image from the test dataset
image_index = random.randint(0, len(test_images) - 1)

# Selected the random image and its corresponding label
selected_image = test_images[image_index]
selected_label = np.argmax(test_labels[image_index])

# Display the random image
plt.imshow(selected_image)
plt.axis('off')
plt.title(f'Actual Label: {selected_label}')
plt.show()

# Make a prediction on the selected image
prediction = model.predict(np.expand_dims(selected_image, axis=0))
predicted_label = np.argmax(prediction)
confidence = np.max(prediction)

# Mapping from label indices to class names
class_names = ['0 - Angry', '1 - Disgust', '2 - Fear', '3 - Happy', '4 - Neutral',
               '5 - Sad', '6 - Surprise']

# Display the model's prediction
print(f"Predicted Label: {class_names[predicted_label]}, Confidence: {confidence}")
